<a href="https://colab.research.google.com/github/Kimtaewookdev/Sec2_project/blob/main/AI_13_%EA%B9%80%ED%83%9C%EC%9A%B1_section2_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AI_13 Section2 Project 
# League of Legends 승패 예측

### 1. 데이터 선정 이유 및 문제 정의



In [ ]:
!pip install category_encoders==2.*
!pip install eli5
!pip install --upgrade pandas-profiling
!pip install pdpbox
!pip install shap
!pip install --upgrade xgboost
!pip install --upgrade jinja2

In [65]:
import xgboost
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pandas_profiling import profile_report

warnings.filterwarnings("ignore")
xgboost.set_config(verbosity=0)

In [3]:
df = pd.read_csv('/content/drive/MyDrive/sec2 ds/lol_na_s11_highelo.csv')

# 불필요 칼럼 제거

dc1 = df.columns[['veteran' in x for x in df.columns]].tolist()
dc2 = df.columns[['inactive' in x for x in df.columns]].tolist()
dc3 = df.columns[['freshblood' in x for x in df.columns]].tolist()
dc4 = df.columns[['hotstreak' in x for x in df.columns]].tolist()
dc5 = df.columns[['wins' in x for x in df.columns]].tolist()
dc6 = df.columns[['losses' in x for x in df.columns]].tolist()
dc7 = df.columns[['avg_game_time' in x for x in df.columns]].tolist()

dc = dc1 + dc2 + dc3 + dc4 + dc5 + dc6 + dc7

wr = df.copy()

wr.drop(dc, axis = 1, inplace = True)

In [4]:
wr.duplicated(['gameid']).value_counts()

False    2088
dtype: int64

In [5]:
wr.drop('gameid', axis = 1, inplace = True)

In [6]:
# 티어, 랭크, lp = lp 포인트로 정리
# 결측치는 0 으로 처리

def trans_rank(wr):
  team = [100, 200]
  position = ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']
  for t in team:
    for p in position:
      col = f'{t}_' + p + '_rank'
      wr[col][(wr[col] == 'I')] = -100
      wr[col][(wr[col] == 'II')] = -200
      wr[col][(wr[col] == 'III')] = -300
      wr[col][(wr[col] == 'IV')] = -400
      wr[col].fillna(0, inplace = True)

def trans_tier(wr):
  team = [100, 200]
  position = ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']
  for t in team:
    for p in position:
      col = f'{t}_' + p + '_tier'
      wr[col][(wr[col] == 'CHALLENGER')] = 100
      wr[col][(wr[col] == 'GRANDMASTER')] = 100
      wr[col][(wr[col] == 'MASTER')] = 100
      wr[col][(wr[col] == 'DIAMOND')] = 0
      wr[col][(wr[col] == 'PLATINUM')] = -400
      wr[col][(wr[col] == 'GOLD')] = -800
      wr[col][(wr[col] == 'SILVER')] = -1200
      wr[col].fillna(0, inplace = True)

def trans_lp(wr):
  team = [100, 200]
  position = ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']
  for t in team:
    for p in position:
      tier = f'{t}_' + p + '_tier'
      rank = f'{t}_' + p + '_rank'
      lp = f'{t}_' + p + '_lp'
      wr[lp].fillna(0, inplace = True)
      wr[lp] = wr[tier] + wr[rank] + wr[lp]
      wr.drop([tier, rank], axis = 1, inplace = True)

trans_rank(wr)
trans_tier(wr)
trans_lp(wr)

wr.shape

(2088, 141)

In [7]:
# 최근 10 게임 포지션 칼럼을 가장 많이 플레이한 포지션 하나로 정리

def mode_is(list):
  return stats.mode(list)[0][0]

def most_position(wr):
  team = [100, 200]
  position = ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']
  for t in team:
    for p in position:
      col = f'{t}_' + p + '_positions_played'
      most_col = f'{t}_' + p + '_most_played'
      wr[col] = wr[col].str.split().apply(mode_is)
      wr.rename(columns={col : most_col}, inplace = True)

most_position(wr)

In [8]:
wr

,winning_team,100_TOP_most_played,100_TOP_avg_kda,100_TOP_avg_kp,100_TOP_avg_vision,100_TOP_avg_cs,100_TOP_goldpm,100_TOP_dmgpm,100_TOP_dmg_to_obj_pm,100_TOP_wr,...,200_UTILITY_avg_cs,200_UTILITY_goldpm,200_UTILITY_dmgpm,200_UTILITY_dmg_to_obj_pm,200_UTILITY_wr,200_UTILITY_lp,200_UTILITY_role_total_played,200_UTILITY_role_wr,200_UTILITY_champ_games_played,200_UTILITY_champ_winrate
0,200,TOP,2.344102,0.421483,19.9,7.399556,450.255676,660.584402,430.506338,0.5,...,5.102581,374.791529,439.441430,364.530774,0.4,825.0,616.0,0.524351,25,0.360000
1,200,TOP,2.344102,0.421483,19.9,7.399556,450.255676,660.584402,430.506338,0.5,...,2.770318,326.858657,471.254417,135.872792,0.1,651.0,941.0,0.547290,25,0.520000
2,200,TOP,2.344102,0.421483,19.9,7.399556,450.255676,660.584402,430.506338,0.5,...,8.155022,502.503639,979.497817,501.419214,0.6,1387.0,28.0,0.500000,6,0.833333
3,200,TOP,1.912560,0.368992,16.5,6.744090,424.682704,688.888428,230.514309,0.4,...,5.851240,380.521297,708.835346,175.674507,0.5,1176.0,30.0,0.566667,69,0.652174
4,100,TOP,3.170368,0.357680,18.0,6.643375,404.560602,473.586402,530.377587,0.5,...,1.418175,289.282492,261.333975,72.215072,0.3,1096.0,979.0,0.539326,733,0.567531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,100,TOP,2.217042,0.408708,19.4,6.754376,427.093702,716.603167,435.780450,0.7,...,0.723918,277.763727,226.222337,89.173495,0.4,-85.0,40.0,0.525000,40,0.550000
2084,200,TOP,5.860595,0.465063,29.1,7.977548,485.142957,852.225925,340.270128,0.6,...,1.817592,309.270887,263.628697,183.463934,0.4,136.0,33.0,0.484848,2,1.000000
2085,200,MIDDLE,4.081071,0.492928,19.1,5.714827,433.344710,691.186955,142.999621,0.5,...,2.228251,318.574787,446.464098,198.893109,0.7,-77.0,16.0,0.500000,2,0.500000
2086,200,TOP,3.063929,0.335034,31.8,6.572781,405.116011,575.201431,199.967679,0.6,...,4.548207,366.326269,696.190033,138.391942,0.6,-56.0,6.0,0.666667,1,1.000000


In [54]:
# 데이터를 100(Blue), 200(Red) 팀으로 나누어서 한 팀의 정보만 담은 데이터셋으로 변환

wr_col100 = wr.columns[['100' in x for x in wr.columns]].tolist()
wr_col200 = wr.columns[['200' in x for x in wr.columns]].tolist()

wr_blue = wr[wr_col100].copy()
wr_red = wr[wr_col200].copy()

wt = 'winning_team'

wr_blue[wt] = wr[wt]
wr_red[wt] = wr[wt]

wr_blue['Team'] = 'Blue'
wr_red['Team'] = 'Red'

# 'Result' column은 0 패배 1 승리

wr_blue['Result'] = 0
wr_blue['Result'][wr_blue[wt] == 100] = 1

wr_red['Result'] = 0
wr_red['Result'][wr_red[wt] == 200] = 1

wr_blue.drop(wt, axis = 1, inplace = True)
wr_red.drop(wt, axis = 1, inplace = True)

wr_blue

,100_TOP_most_played,100_TOP_avg_kda,100_TOP_avg_kp,100_TOP_avg_vision,100_TOP_avg_cs,100_TOP_goldpm,100_TOP_dmgpm,100_TOP_dmg_to_obj_pm,100_TOP_wr,100_TOP_lp,...,100_UTILITY_dmgpm,100_UTILITY_dmg_to_obj_pm,100_UTILITY_wr,100_UTILITY_lp,100_UTILITY_role_total_played,100_UTILITY_role_wr,100_UTILITY_champ_games_played,100_UTILITY_champ_winrate,Team,Result
0,TOP,2.344102,0.421483,19.9,7.399556,450.255676,660.584402,430.506338,0.5,765.0,...,563.949624,98.107399,0.3,374.0,1947.0,0.511556,3331,0.507355,Blue,0
1,TOP,2.344102,0.421483,19.9,7.399556,450.255676,660.584402,430.506338,0.5,765.0,...,305.332855,103.118608,0.7,820.0,1327.0,0.514695,225,0.542222,Blue,0
2,TOP,2.344102,0.421483,19.9,7.399556,450.255676,660.584402,430.506338,0.5,765.0,...,554.237533,147.054926,0.5,601.0,119.0,0.546218,1,0.000000,Blue,0
3,TOP,1.912560,0.368992,16.5,6.744090,424.682704,688.888428,230.514309,0.4,1032.0,...,216.193534,46.067285,0.7,876.0,462.0,0.582251,25,0.600000,Blue,0
4,TOP,3.170368,0.357680,18.0,6.643375,404.560602,473.586402,530.377587,0.5,694.0,...,925.302577,219.550161,0.2,719.0,28.0,0.571429,20,0.600000,Blue,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,TOP,2.217042,0.408708,19.4,6.754376,427.093702,716.603167,435.780450,0.7,0.0,...,284.884047,128.074746,0.7,1.0,49.0,0.591837,11,0.636364,Blue,1
2084,TOP,5.860595,0.465063,29.1,7.977548,485.142957,852.225925,340.270128,0.6,26.0,...,265.708437,100.961538,0.6,100.0,64.0,0.562500,1,0.000000,Blue,0
2085,MIDDLE,4.081071,0.492928,19.1,5.714827,433.344710,691.186955,142.999621,0.5,0.0,...,193.918540,46.460731,0.3,-62.0,82.0,0.451220,45,0.533333,Blue,0
2086,TOP,3.063929,0.335034,31.8,6.572781,405.116011,575.201431,199.967679,0.6,-232.0,...,255.766753,23.175278,0.5,-32.0,23.0,0.521739,4,0.000000,Blue,0


In [60]:
# colunm 명 변경

def remove_100(x):
  if '100' in x:
    x.remove('100')
  str = '_'.join(x)
  return str

col_united = wr_blue.columns.str.split('_').map(remove_100)

wr_blue.columns = col_united
wr_red.columns = col_united

# 병합

qw = pd.concat([wr_blue, wr_red], ignore_index=True)
qw

,TOP_most_played,TOP_avg_kda,TOP_avg_kp,TOP_avg_vision,TOP_avg_cs,TOP_goldpm,TOP_dmgpm,TOP_dmg_to_obj_pm,TOP_wr,TOP_lp,...,UTILITY_dmgpm,UTILITY_dmg_to_obj_pm,UTILITY_wr,UTILITY_lp,UTILITY_role_total_played,UTILITY_role_wr,UTILITY_champ_games_played,UTILITY_champ_winrate,Team,Result
0,TOP,2.344102,0.421483,19.9,7.399556,450.255676,660.584402,430.506338,0.5,765.0,...,563.949624,98.107399,0.3,374.0,1947.0,0.511556,3331,0.507355,Blue,0
1,TOP,2.344102,0.421483,19.9,7.399556,450.255676,660.584402,430.506338,0.5,765.0,...,305.332855,103.118608,0.7,820.0,1327.0,0.514695,225,0.542222,Blue,0
2,TOP,2.344102,0.421483,19.9,7.399556,450.255676,660.584402,430.506338,0.5,765.0,...,554.237533,147.054926,0.5,601.0,119.0,0.546218,1,0.000000,Blue,0
3,TOP,1.912560,0.368992,16.5,6.744090,424.682704,688.888428,230.514309,0.4,1032.0,...,216.193534,46.067285,0.7,876.0,462.0,0.582251,25,0.600000,Blue,0
4,TOP,3.170368,0.357680,18.0,6.643375,404.560602,473.586402,530.377587,0.5,694.0,...,925.302577,219.550161,0.2,719.0,28.0,0.571429,20,0.600000,Blue,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4171,MIDDLE,7.217500,0.485764,19.5,6.350779,432.854070,601.313954,180.336310,1.0,1.0,...,226.222337,89.173495,0.4,-85.0,40.0,0.525000,40,0.550000,Red,0
4172,TOP,3.712962,0.353581,16.0,7.049641,418.459432,612.172544,312.250599,0.7,233.0,...,263.628697,183.463934,0.4,136.0,33.0,0.484848,2,1.000000,Red,1
4173,TOP,4.283333,0.401685,20.9,6.491852,420.580633,517.722420,453.867765,0.5,-131.0,...,446.464098,198.893109,0.7,-77.0,16.0,0.500000,2,0.500000,Red,1
4174,TOP,2.171414,0.397915,21.8,7.569480,433.896037,775.236747,318.335049,0.4,234.0,...,696.190033,138.391942,0.6,-56.0,6.0,0.666667,1,1.000000,Red,1


In [10]:
print(wr.isna().sum().tolist())

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, 14, 0, 75, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 15, 0, 145, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 15, 0, 36, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 12, 0, 236, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 16, 0, 31, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 17, 0, 82, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 17, 0, 150, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 15, 0, 43, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 11, 0, 245, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 12, 0, 29]


In [ ]:
pr = qw.profile_report()
pr.to_file('pr_report.html')